# Get Artifacts

In [1]:
import sys
import torch

import os; os.environ['CUDA_VISIBLE_DEVICES']='0'
torch.cuda.set_device('cuda:0')
sys.path.append("/home/hamel/hamel_fork/edu/mlops-001/lesson3/")


In [2]:
#|export
import wandb, params
import pandas as pd
from pathlib import Path
from fastai.vision.all import load_learner
from train import prepare_df

In [3]:
#|export
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_data_split(model_registry_artifact):
    "Get the validation and test dataset from the artifacts associated with the model from the registry."
    parent_run_artifacts = model_registry_artifact.logged_by().used_artifacts()
    datasets_used = [a for a in parent_run_artifacts if a.type == 'split_data']
    assert len(datasets_used) == 1, f'Only expected one input artifact with type == `split_data`, but found {len(datasets_used)}'
    data_artifact = datasets_used[0]
    artifact_dir = Path(data_artifact.download())
    return prepare_df(artifact_dir), prepare_df(artifact_dir, is_test=True)
    

def get_eval_artifacts(run, art_name='av-team/model-registry/BDD Semantic Segmentation:staging', art_type='model'):
    "Return the learner and the data split with the holdout set."
    artifact = run.use_artifact(art_name, type=art_type)
    artifact_dir = Path(artifact.download())
    model = load_learner(artifact_dir/'fastai_model.pkl')
    model.cuda()
    train_df, test_df = get_data_split(artifact)
   
    return model, train_df, test_df

In [4]:
#|export
run = wandb.init(project=params.WANDB_PROJECT, 
                 entity=params.ENTITY, 
                 job_type="evaluation")

fastai_model, train_df, test_df = get_eval_artifacts(run)

wandb: Currently logged in as: hamelsmu (av-team). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact BDD Semantic Segmentation:staging, 237.50MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact bdd_simple_1k_split:v4, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:0.4


In [5]:
assert torch.cuda.is_available()
assert next(fastai_model.model.parameters()).is_cuda

In [6]:
train_df.Stage.value_counts()

train    800
valid    100
Name: Stage, dtype: int64

In [7]:
test_df.Stage.value_counts()

test    100
Name: Stage, dtype: int64

In [8]:
#|export 
valid_dls = fastai_model.dls.test_dl(train_df[train_df.Stage == 'valid'], device=torch.device('cuda:0'))
val_metrics = fastai_model.validate(dl=valid_dls)

KeyboardInterrupt: 

In [10]:
from nbdev.export import nb_export
nb_export('evaluate.ipynb', lib_path='.', name='evaluate')

ImportError: cannot import name 'nb_export' from 'nbdev.export' (/home/hamel/anaconda3/lib/python3.9/site-packages/nbdev/export.py)